# Bulk RNA-Seq Analysis Training Demo

## Overview

This short tutorial demonstrates how to run an RNA-Seq workflow using a subsampling Caenorhabditis elegans dataset. Steps in the workflow include read trimming, read QC, read mapping, and counting mapped reads per gene to quantitate gene expression. This tutorial workflow uses truncated and partial run data from the Rochester JD et al. project.

This tutorial can take over 17 minutes to run the code fully, with a `ml.t3.medium` instance, because of the limited memory we are using a subsampled dataset from Rochester JD et al. from chr I and 1,000,000 reads each sample. This is part of the reason we have a short and easy introductory tutorial, but provide a longer more full tutorial for those interested.

## STEP 1: Getting Started

<div class="alert alert-block alert-warning"> NOTE: This Jupyter Notebook was developed to run within a customized container on AWS with all software and tools pre-configured. If running without this customized container, you will need to install tools using the Miniforge environment setup instructions below before moving on to Step 2.</div>

### Without Container: Install Miniforge and Workflow Tools

Miniforge is a lightweight Conda distribution that offers a streamlined installation process and efficient package management. It provides access to a vast repository of packages.

Conda packages and environments are useful for several reasons. Conda packages contain metadata. This metadata includes information about what other programs the given software needs to be installed, in order to run. When installing a package with Conda, those other packages are automatically also installed. In this way, the user does not have to worry about manually installing each dependency. This makes installation quick and simple.

These packages are installed inside of environments, which are simply folders within the local installation of Conda. This has several benefits. Local installation means easier installation for non-admin users who may not have access to all system directories. Each environment can hold specific software with specific versions, and it easy to swap to different environments. In addition, the environments themselves are portable, as each environment contains a manifest on how to recreate that environment.

Miniforge itself is a Conda package manager, this means it requires Conda in order to work. It is used to install and update Conda packages, which it gets from a ‘channel’, or repository. It is an alternative to the native Conda package manager. It is often used for reasons of speed.

Bioconda is a ‘channel’, or repository, that the Mambaforge package manager can download packages from. It is a repository of Conda packages that are related to biology. These packages are versions of popular biology software that are curated and uploaded by contributing users.

The following code performs these steps:
- Downloads Miniforge or Mambaforge (you can use either based on preference)
- Installs Miniforge (or Mambaforge) - no need to install conda since mamba will be available immediately
- Using miniforge and bioconda, installs the tools that will be used in this tutorial

<div class="alert alert-block alert-info">Tip: If using the Miniforge install, run the following code cells by removing the %%script false command. </div>

In [1]:
#%%script false --no-raise-error
# Download Miniforge or Mambaforge (you can use either based on preference)
!curl -L -O https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-$(uname)-$(uname -m).sh > /dev/null 2>&1

# Install Miniforge (or Mambaforge) - no need to install conda since mamba will be available immediately
!bash Miniforge3-$(uname)-$(uname -m).sh -b -u -p $HOME/miniforge > /dev/null 2>&1
!date +"%T"

11:14:13


Next, using miniforge and bioconda, install the tools that will be used in this tutorial.

In [2]:
#%%script false --no-raise-error
# Update PATH to point to the Miniforge (or Mambaforge) bin files
import os
os.environ["PATH"] += os.pathsep + os.environ["HOME"]+"/miniforge/bin"

# Now use mamba directly to install your software packages
!mamba install -y -c conda-forge -c bioconda trimmomatic fastqc multiqc sql-magic entrez-direct gffread parallel-fastq-dump sra-tools pyathena samtools star rsem subread pigz -y > /dev/null

warning  libmamba Could not parse mod/etag header
warning  libmamba Could not parse mod/etag header
warning  libmamba Could not parse mod/etag header
warning  libmamba Could not parse mod/etag header
warning  libmamba Could not parse mod/etag header
warning  libmamba Could not parse mod/etag header


---------------------------------------
## If running from a container, as noted above, start with <b> STEP 2 </b> below:
## STEP 2: Define Threads & Setup Directory Structures

Create a set of directories to store the reads, reference sequence files, and output files.

In [3]:
!cd $HOMEDIR
!echo $PWD
!mkdir -p data_1
!mkdir -p data_1/trunc_rawfastq
!mkdir -p data_1/trimmed
!mkdir -p data_1/fastqc
!mkdir -p data_1/fastqc_samples/
!mkdir -p data_1/reference
!mkdir -p data_1/raw_fastq
!mkdir -p data_1/aligned_bam
!mkdir -p data_1/rsem_reference/celegans_rsem_reference
!mkdir -p data_1/rsem_output
!mkdir -p data_1/reference/STAR_index
!mkdir -p data_1/gene_counts
!mkdir -p data_1/multiqc_samples

/home/ec2-user/SageMaker/Bulk-and-Single-Cell-RNAseq/Bulk_RNA-Seq_Tutorials/Bulk_RNA-Seq_C_elegans


Specify the number of available threads based on the VM.

This is useful for later tools such as trimmomatic, or STAR.

In [4]:
import multiprocessing
import os

num_cores = multiprocessing.cpu_count()
THREADS = max(1, num_cores - 1)

print("Number of threads:", THREADS)
os.environ["THREADS"] = str(THREADS)

Number of threads: 15


## STEP 3: Downloading relevant files



### STEP 3.1: Finding run accession numbers.


This code retrieves the SRR IDs associated with project PRJNA625528 from the NCBI database and saves them to a .txt file

In [5]:
!esearch -db sra -query "PRJNA625528" | efetch -format runinfo | cut -d',' -f1 | tail -n +2 > all_accs.txt
!grep -E "SRR11550221|SRR11550223|SRR11550225|SRR11550227|SRR11550229|SRR11550231" all_accs.txt > accs.txt
!cat accs.txt

SRR11550221
SRR11550223
SRR11550225
SRR11550227
SRR11550229
SRR11550231


### STEP 3.2: Download subsampling FASTQ files from S3 bucket

In order for this tutorial to run quickly, we will only analyze 1,000,000 reads from each of the six samples in PRJNA625528. These subsampled files have been posted on a publicly accessible S3 bucket.

The code will connect to the S3 bucket and download the paired-end FASTQ files for each SRR.

In [6]:
#Load SRR IDs from accs.txt (if available)
with open('accs.txt', 'r') as f:
    accs = [line.strip() for line in f.readlines()]

In [7]:
for acc in accs:
    !wget -P data_1/raw_fastq/ https://nigms-sandbox.s3.us-east-1.amazonaws.com/bulk-scRNAseq/fastqfiles/subsampledchr1_celegans_{acc}.fastq

--2025-05-12 11:16:43--  https://nigms-sandbox.s3.us-east-1.amazonaws.com/bulk-scRNAseq/fastqfiles/subsampledchr1_celegans_SRR11550221.fastq
Resolving nigms-sandbox.s3.us-east-1.amazonaws.com (nigms-sandbox.s3.us-east-1.amazonaws.com)... 3.5.31.21, 52.216.58.26, 16.182.104.98, ...
Connecting to nigms-sandbox.s3.us-east-1.amazonaws.com (nigms-sandbox.s3.us-east-1.amazonaws.com)|3.5.31.21|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 177726603 (169M) [binary/octet-stream]
Saving to: ‘data_1/raw_fastq/subsampledchr1_celegans_SRR11550221.fastq’

subsampledchr1_cele 100%[===================>] 169.49M  36.9MB/s    in 4.5s    

2025-05-12 11:16:48 (37.3 MB/s) - ‘data_1/raw_fastq/subsampledchr1_celegans_SRR11550221.fastq’ saved [177726603/177726603]

--2025-05-12 11:16:48--  https://nigms-sandbox.s3.us-east-1.amazonaws.com/bulk-scRNAseq/fastqfiles/subsampledchr1_celegans_SRR11550223.fastq
Resolving nigms-sandbox.s3.us-east-1.amazonaws.com (nigms-sandbox.s3.us-east-1

### STEP 3.3: Download reference genome and annotation files that will be used by STAR and RSEM

This step downloads and unzips reference files for the C. elegans genome and annotations needed by STAR. These reference files will be used by STAR to align RNA-seq reads during the analysis.

In [8]:
!wget https://nigms-sandbox.s3.us-east-1.amazonaws.com/bulk-scRNAseq/reference/celegans_genome.fa -O data_1/reference/celegans_genome.fa
!wget https://nigms-sandbox.s3.us-east-1.amazonaws.com/bulk-scRNAseq/reference/celegans_genomic.gtf -O data_1/reference/celegans_annotation.gtf
!wget https://nigms-sandbox.s3.us-east-1.amazonaws.com/bulk-scRNAseq/reference/celegans_feature_table.txt -O data_1/reference/celegans_feature_table.txt

--2025-05-12 11:17:12--  https://nigms-sandbox.s3.us-east-1.amazonaws.com/bulk-scRNAseq/reference/celegans_genome.fa
Resolving nigms-sandbox.s3.us-east-1.amazonaws.com (nigms-sandbox.s3.us-east-1.amazonaws.com)... 52.217.232.210, 3.5.14.37, 52.217.42.168, ...
Connecting to nigms-sandbox.s3.us-east-1.amazonaws.com (nigms-sandbox.s3.us-east-1.amazonaws.com)|52.217.232.210|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 101957874 (97M) [binary/octet-stream]
Saving to: ‘data_1/reference/celegans_genome.fa’

data_1/reference/ce 100%[===================>]  97.23M  41.1MB/s    in 2.4s    

2025-05-12 11:17:15 (41.1 MB/s) - ‘data_1/reference/celegans_genome.fa’ saved [101957874/101957874]

--2025-05-12 11:17:15--  https://nigms-sandbox.s3.us-east-1.amazonaws.com/bulk-scRNAseq/reference/celegans_genomic.gtf
Resolving nigms-sandbox.s3.us-east-1.amazonaws.com (nigms-sandbox.s3.us-east-1.amazonaws.com)... 52.217.161.34, 3.5.0.70, 52.216.205.174, ...
Connecting to nigms-sa

In [9]:
# Use samtools to extract only chr1 from the reference genome
!samtools faidx data_1/reference/celegans_genome.fa I > data_1/reference/celegans_genome_chr1.fa

# Subset the GTF file to include only chr1 annotations
!grep "^I" data_1/reference/celegans_annotation.gtf > data_1/reference/celegans_annotation_chr1.gtf

### STEP 3.4: Copy data file for Trimmomatic


One of trimmomatics functions is to trim sequence machine specific adapter sequences. These are usually within the trimmomatic installation directory in a folder called adapters.

Directories of packages within conda installations can be confusing, so in the case of using conda with trimmomatic, it may be easier to simply download or create a file with the relevant adapter sequencecs and store it in an easy to find directory.

In [10]:
!wget https://nigms-sandbox.s3.us-east-1.amazonaws.com/bulk-scRNAseq/reference/TruSeq3-SE.fa -O data_1/trimmed/TruSeq3-SE.fa

--2025-05-12 11:17:23--  https://nigms-sandbox.s3.us-east-1.amazonaws.com/bulk-scRNAseq/reference/TruSeq3-SE.fa
Resolving nigms-sandbox.s3.us-east-1.amazonaws.com (nigms-sandbox.s3.us-east-1.amazonaws.com)... 54.231.228.2, 52.217.126.154, 3.5.12.136, ...
Connecting to nigms-sandbox.s3.us-east-1.amazonaws.com (nigms-sandbox.s3.us-east-1.amazonaws.com)|54.231.228.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 120 [binary/octet-stream]
Saving to: ‘data_1/trimmed/TruSeq3-SE.fa’

data_1/trimmed/TruS 100%[===================>]     120  --.-KB/s    in 0s      

2025-05-12 11:17:23 (5.25 MB/s) - ‘data_1/trimmed/TruSeq3-SE.fa’ saved [120/120]



## STEP 4: Run Trimmomatic

Trimmomatic will trim off any adapter sequences or low quality sequence it detects in the FASTQ files.

Trimmomatic takes an input, in this case a forward and reverse FASTQ file, and the user names the outputs. In this case the first two outputs are the ‘paired’ and ‘orphaned’ trimmed files for the forward reads. The second two are the ‘paired’ and ‘orphaned’ trimmed files for the reverse reads.

Paired in this case means the forward and reverse sequences were able to be aligned to each other after trimming. Orphaned, or unpaired, typically means one of the reads was discarded as a result of trimming, and only the forward or reverse read survived.

Here we take only the file with paired reads, as there are only a few orphaned reads, and including orphaned reads can complicate different downstream analyses. Unless there is a significant amount of them, or a specific reason to use them, it is generally easier to discard unpaired reads. Also in the interest of simplicity and speed, we proceed in further steps with just using the paired-end reads of just the forward-end read files, which in this context is sufficient, however in different contexts, using both forward and reverse is often preferable.

The last part of the command specifies how the trimming is done: `ILLUMINACLIP:TruSeq3-PE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36`

<u>In greater detail</u>:

`ILLUMINACLIP:TruSeq3-PE.fa` refers to which adapters should be cut from the reads.

`2:30:10:2` refers to various metrics, which are recommended defaults.

`2` refers to the seed mismatch. This refers to the amount of mismatches a 'seed' may have in aligning to a possible adapter.

`30` refers to the palindrome clip threshold. This refers to the similarity score. If forward and reverse reads, after having an adapter attached to them, are greater than this score, trimming of adapter fragments will be performed. Forward reads are clipped, and reverse reads dropped.

`10` refers to the simple clip threshold. The basic, and alternative method to palindromic searching for adapters. Adapters are tested against reads and if sufficiently matched (above the threshold, in this case, 10), they are clipped.

`2` refers to the minimum adapter fragment length in palindrome mode.

`LEADING:3` refers to trimming bases from the start of a read. Bases at the start of a read will continue to be trimmed, sequentially, as long as the bases remain below a PHRED score of 3.

`TRAILING:3` refers to the same as above, but at the end of a read.

`MINLEN:36`, the read is discarded if below this length.

Greater information about parameters can be found in the trimmomatic documentation.

In [11]:
!cat accs.txt | xargs -I {} \
trimmomatic SE -threads $THREADS \
data_1/raw_fastq/subsampledchr1_celegans_{}.fastq data_1/trimmed/{}_trimmed.fastq \
ILLUMINACLIP:data_1/trimmed/TruSeq3-SE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36

TrimmomaticSE: Started with arguments:
 -threads 15 data_1/raw_fastq/subsampledchr1_celegans_SRR11550221.fastq data_1/trimmed/SRR11550221_trimmed.fastq ILLUMINACLIP:data_1/trimmed/TruSeq3-SE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36
Using Long Clipping Sequence: 'AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTA'
Using Long Clipping Sequence: 'AGATCGGAAGAGCACACGTCTGAACTCCAGTCAC'
ILLUMINACLIP: Using 0 prefix pairs, 2 forward/reverse sequences, 0 forward only sequences, 0 reverse only sequences
Quality encoding detected as phred33
Input Reads: 1000000 Surviving: 1000000 (100.00%) Dropped: 0 (0.00%)
TrimmomaticSE: Completed successfully
TrimmomaticSE: Started with arguments:
 -threads 15 data_1/raw_fastq/subsampledchr1_celegans_SRR11550223.fastq data_1/trimmed/SRR11550223_trimmed.fastq ILLUMINACLIP:data_1/trimmed/TruSeq3-SE.fa:2:30:10:2:keepBothReads LEADING:3 TRAILING:3 MINLEN:36
Using Long Clipping Sequence: 'AGATCGGAAGAGCGTCGTGTAGGGAAAGAGTGTA'
Using Long Clipping Sequence: 'AGATCGGAAG

## STEP 5: Run FastQC
FastQC is an invaluable tool that allows you to evaluate whether there are problems with a set of reads. For example, it will provide a report of whether there is any bias in the sequence composition of the reads.

Because jupyter is at its core a python editor, we can use python code and html support to display results in-line.

FastQC looks for different characteristics of quality in reads. It is very rare that every metric will pass. In many cases, they serve as warnings, which should be compared to the context of the experiment. For instance, here, per base sequence content, sequence length distribution, sequence duplication levels, and overrepresented sequences all throw warnings. Per base sequence content routinely fails in RNA-sequencing in the first 15~ or so bases due to biased fragmentation. In most of our samples, this is where we see the failure (20% or more difference between A/T or G/C), and so is not unexpected. The overrepresented sequences can be BLASTed to show the majority of them are ribosomal RNA. Ribosomal RNA contamination is also common and will not be indexed later, and so not a large concern. Other metrics look good.

In [12]:
# Run FastQC
!cat accs.txt | xargs -P $THREADS -I {} fastqc data_1/trimmed/{}_trimmed.fastq -o data_1/fastqc_samples/

null
null
null
null
null
null
Started analysis of SRR11550231_trimmed.fastq
Started analysis of SRR11550227_trimmed.fastq
Started analysis of SRR11550229_trimmed.fastq
Started analysis of SRR11550225_trimmed.fastq
Started analysis of SRR11550223_trimmed.fastq
Started analysis of SRR11550221_trimmed.fastq
Approx 5% complete for SRR11550231_trimmed.fastq
Approx 5% complete for SRR11550227_trimmed.fastq
Approx 5% complete for SRR11550221_trimmed.fastq
Approx 5% complete for SRR11550229_trimmed.fastq
Approx 5% complete for SRR11550225_trimmed.fastq
Approx 5% complete for SRR11550223_trimmed.fastq
Approx 10% complete for SRR11550231_trimmed.fastq
Approx 10% complete for SRR11550221_trimmed.fastq
Approx 10% complete for SRR11550227_trimmed.fastq
Approx 10% complete for SRR11550229_trimmed.fastq
Approx 10% complete for SRR11550225_trimmed.fastq
Approx 10% complete for SRR11550223_trimmed.fastq
Approx 15% complete for SRR11550231_trimmed.fastq
Approx 15% complete for SRR11550221_trimmed.fastq


In [13]:
from IPython.display import IFrame
IFrame(src='./data_1/fastqc_samples/SRR11550221_trimmed_fastqc.html', width=800, height=600)

## STEP 6: Run MultiQC
MultiQC reads in the FastQC reports and generate a compiled report for all the analyzed FASTQ files.

Being able to use python with bash also means we can seamlessly use popular python packages, such as pandas, to interact with or view the files we create.

In [14]:
#!multiqc -f data_1/fastqc_samples/
!multiqc -f -o data_1/multiqc_samples/ data_1/fastqc_samples/


/// ]8;id=855396;https://multiqc.info\MultiQC]8;;\ 🔍 v1.28

       file_search | Search path: /home/ec2-user/SageMaker/Bulk-and-Single-Cell-RNAseq/Bulk_RNA-Seq_Tutorials/Bulk_RNA-Seq_C_elegans/data_1/fastqc_samples
         searching | ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100% 12/12  pm  
            fastqc | Found 6 reports
     write_results | Data        : data_1/multiqc_samples/multiqc_data
     write_results | Report      : data_1/multiqc_samples/multiqc_report.html
           multiqc | MultiQC complete


## STEP 7: Preparing the STAR-Compatible RSEM Reference

The provided code snippet demonstrates the process of preparing a reference genome and annotation files for RNA-Seq analysis using RSEM and STAR. The initial command, rsem-prepare-reference, creates a reference index for the C. elegans genome based on the provided GTF annotation file (celegans_annotation.gtf). This index, along with the reference genome sequence (celegans_genome.fa), is essential for quantifying gene and isoform expression using RSEM. The --star option ensures compatibility with STAR, a popular transcriptome aligner.

However, the subsequent STAR genome generation step encounters a warning due to an excessively large --genomeSAindexNbases parameter. This parameter controls the size of the genome's suffix array index, which is used for efficient alignment. If the parameter is too large for the given genome size, it can lead to segmentation faults during the mapping process. To address this issue, the code adjusts the --genomeSAindexNbases parameter to 12, which is more appropriate for the C. elegans genome. This modification ensures that the STAR genome generation process completes successfully without errors.

In [15]:
!rsem-prepare-reference --gtf data_1/reference/celegans_annotation_chr1.gtf --star -p $THREADS data_1/reference/celegans_genome_chr1.fa celegans_reference > /dev/null

The reference contains no transcripts!


In [16]:
!STAR  --runThreadN $THREADS  --runMode genomeGenerate  --genomeDir .  --genomeFastaFiles data_1/reference/celegans_genome_chr1.fa  --sjdbGTFfile data_1/reference/celegans_annotation_chr1.gtf  --sjdbOverhang 100  --outFileNamePrefix celegans_reference  --genomeSAindexNbases 10 > /dev/null


Fatal INPUT FILE error, no exon lines in the GTF file: data_1/reference/celegans_annotation_chr1.gtf
Solution: check the formatting of the GTF file, it must contain some lines with exon in the 3rd column.
          Make sure the GTF file is unzipped.
          If exons are marked with a different word, use --sjdbGTFfeatureExon .

May 12 11:18:08 ...... FATAL ERROR, exiting


## STEP 8: Automated RNA-seq Quantification with RSEM

This code automates the process of quantifying gene expression from RNA-seq data using RSEM. It reads a list of SRR accession numbers from a text file, iterates over each accession, and executes the RSEM command to calculate gene expression levels. The code assumes that the trimmed FASTQ files for each SRR are located in a specified directory (data_1/trimmed/) and uses a previously generated reference index (celegans_reference). The output files are saved in the data_1/rsem_output directory, with filenames based on the corresponding SRR accession number. This script simplifies the workflow for analyzing multiple RNA-seq samples using RSEM.

In [17]:
import os

# Ensure you've set the path to the RSEM binary
with open('accs.txt', 'r') as f:
    srr_accessions = [line.strip() for line in f.readlines()]
    
# Define the output directory
output_dir = "data_1/rsem_output"

for srr in srr_accessions:
    os.system(f"rsem-calculate-expression -p $THREADS --star "
              f"data_1/trimmed/{srr}_trimmed.fastq celegans_reference "
              f"{output_dir}/{srr}celegans > /dev/null")


EXITING because of FATAL ERROR: could not open genome file .//genomeParameters.txt
SOLUTION: check that the path to genome files, specified in --genomeDir is correct and the files are present, and have user read permsissions

May 12 11:18:08 ...... FATAL ERROR, exiting

EXITING because of FATAL ERROR: could not open genome file .//genomeParameters.txt
SOLUTION: check that the path to genome files, specified in --genomeDir is correct and the files are present, and have user read permsissions

May 12 11:18:08 ...... FATAL ERROR, exiting

EXITING because of FATAL ERROR: could not open genome file .//genomeParameters.txt
SOLUTION: check that the path to genome files, specified in --genomeDir is correct and the files are present, and have user read permsissions

May 12 11:18:08 ...... FATAL ERROR, exiting

EXITING because of FATAL ERROR: could not open genome file .//genomeParameters.txt
SOLUTION: check that the path to genome files, specified in --genomeDir is correct and the files are pr

## STEP 9: Report the top 10 most highly expressed genes in the samples

Top 10 most highly expressed genes in each wild-type sample.


In [18]:
import pandas as pd

# Path to RSEM results directory
rsem_results_dir = 'data_1/rsem_output'

# Loop through each file in accs.txt
for srr_id in open('accs.txt'):
    srr_id = srr_id.strip()  # Remove newline character
    rsem_result_file = f'{rsem_results_dir}/{srr_id}celegans.genes.results'

    # Load the RSEM results into a Pandas DataFrame
    df = pd.read_csv(rsem_result_file, sep='\t')

    # Sort the DataFrame by TPM values in descending order and get the top 10 genes
    top_10_genes = df.sort_values(by='TPM', ascending=False).head(10)

    # Print the top 10 genes with their TPM values
    print(f"Top 10 Genes by TPM for {srr_id}:")
    print(top_10_genes[['gene_id', 'TPM']])

FileNotFoundError: [Errno 2] No such file or directory: 'data_1/rsem_output/SRR11550221celegans.genes.results'

Top 10 most highly expressed genes in the double lysogen samples.


## STEP 10: Report the expression of WBGene00004512 for each file

Use `grep` to report the expression in the wild-type sample. The fields in the RSEM `genes.results` file are as follows. The level of expression is reported in the Transcripts Per Million (`TPM`) and number of reads (`NumReads`) fields:  
- `Name`    
- `Length`
- `EffectiveLength`
- `TPM`
- `NumReads`

In [ ]:
import pandas as pd

# Path to RSEM results directory
rsem_results_dir = 'data_1/rsem_output'

# Target gene ID
target_gene = 'WBGene00004512'

# Loop through each file in accs.txt
for srr_id in open('accs.txt'):
    srr_id = srr_id.strip()  # Remove newline character
    rsem_result_file = f'{rsem_results_dir}/{srr_id}celegans.genes.results'

    # Load the RSEM results into a Pandas DataFrame
    df = pd.read_csv(rsem_result_file, sep='\t')

    # Filter for the target gene
    target_gene_data = df[df['gene_id'] == target_gene]

    # Print the target gene's TPM value for the SRR ID
    print(f"TPM for {target_gene} in {srr_id}: {target_gene_data['TPM'].values[0]}")

## <a name="workflow">Additional Workflows</a>

Now that you have read counts per gene, feel free to explore the R workflow which creates plots and analyses using these readcount files, or try other alternate workflows for creating read count files, such as using snakemake.


[Workflow One (Extended):](Tutorial_1B_Extended_celegans.ipynb) An extended version of workflow one. Once you have got your feet wet, you can retry workflow one with this extended version that covers the entire dataset.

[Workflow Two (DEG Analysis):](Tutorial_2_DEG_Analysis_celegans.ipynb) Using Deseq2 and R to conduct clustering and differential gene expression analysis.
